In [2]:
import pandas as pd
from scipy.io import loadmat
from dtaidistance import dtw_ndim
from dtaidistance import dtw_ndim_visualisation
import dtw
from tqdm import tqdm
import numpy as np
from sklearn.metrics.cluster import normalized_mutual_info_score
import matplotlib.pyplot as plt
import sklearn
from sklearn.cluster import SpectralClustering
from sklearn.cluster import DBSCAN
from sklearn_extra.cluster import KMedoids

VMT数据集

In [38]:
data_m= loadmat("../VMT_dataset/CASIA_tjc.mat")
print(data_m.keys())
tracks = data_m["tjc"]
tracks = np.delete(tracks, list(range(0, 1500, 3)) + list(range(1, 1500, 3)), axis=0)
labels = data_m["labels"][:, 0]#.reshape(1, -1)
labels = np.delete(labels, list(range(0, 1500, 3)) + list(range(1, 1500, 3)), axis=0)
labels = labels.flatten()
print(tracks.shape)
print(labels.shape)

dict_keys(['__header__', '__version__', '__globals__', 'labels', 'tjc'])
(500, 1)
(500,)


In [39]:
dis_mat = np.zeros([tracks.shape[0], tracks.shape[0]])
with tqdm(total=tracks.shape[0]) as pbar1:
    for num1,track1 in enumerate(tracks):
        for num2,track2 in enumerate(tracks):
            if num1 < num2:
                dis=dtw_ndim.distance(track1[0], track2[0])
                dis_mat[num1,num2]=dis
                dis_mat[num2,num1]=dis
        pbar1.update(1)
np.save("VMT_dis_mat.npy",dis_mat)

100%|██████████| 500/500 [2:25:04<00:00, 17.41s/it]


In [40]:
#similarity = np.exp(-beta * distance / distance.std())
sim_mat = np.exp(-0.5 * dis_mat / dis_mat.std())
np.save("VMT_sim_mat.npy", sim_mat)

In [43]:
dis_mat=np.load("VMT_dis_mat.npy")
sim_mat=np.load("VMT_sim_mat.npy")

聚类算法实现

In [44]:
sse_max = -1
best_k = 0
sse_list = []
for k in range(2, 21):
    res = SpectralClustering(n_clusters=k, affinity='precomputed').fit(sim_mat)
    tmp = sklearn.metrics.silhouette_score(X=dis_mat, labels=res.labels_, metric='precomputed' )
    sse_list.append(tmp)
    if tmp > sse_max:
        sse_max = tmp
        best_k = k
best_res = SpectralClustering(n_clusters=best_k, affinity='precomputed').fit(sim_mat)
nmi = sklearn.metrics.normalized_mutual_info_score(labels_true=labels, labels_pred=best_res.labels_)
ari = sklearn.metrics.adjusted_rand_score(labels_true=labels, labels_pred=best_res.labels_)

print("sse_max:", sse_max)
print("best_k:", best_k)
print("nmi:", nmi)
print("ari:", ari)

sse_max: 0.7508331991258091
best_k: 7
nmi: 0.8315931196253767
ari: 0.5972081267182896


In [49]:
norm_dis_mat = dis_mat / (dis_mat.max() - dis_mat.min()) * 1000.0
#print(norm_dis_mat)
best_eps = -1
sse_max = -1
sse_list = []
for r in range(20, 100):
    #print(r)
    res = DBSCAN(eps=r, metric='precomputed').fit(norm_dis_mat)
    #print(res.labels_)
    tmp = sklearn.metrics.silhouette_score(X=norm_dis_mat, labels=res.labels_, metric='precomputed' )
    sse_list.append(tmp)
    if tmp > sse_max:
        sse_max = tmp
        best_eps = r
nmi = sklearn.metrics.normalized_mutual_info_score(labels_true=labels, labels_pred=res.labels_)
ari = sklearn.metrics.adjusted_rand_score(labels_true=labels, labels_pred=res.labels_)
print("sse_max:", sse_max)
print("best_eps:", best_eps)
print("nmi:", nmi)
print("ari:", ari)

sse_max: 0.7873347863092436
best_eps: 47
nmi: 0.6346994127650502
ari: 0.28554204205826333


In [50]:
sse_max = -1
best_k = 0
sse_list = []
for k in range(2, 21):
    res = KMedoids(n_clusters=k, metric='precomputed').fit(dis_mat)
    tmp = sklearn.metrics.silhouette_score(X=dis_mat, labels=res.labels_, metric='precomputed' )
    sse_list.append(tmp)
    if tmp > sse_max:
        sse_max = tmp
        best_k = k
best_res = KMedoids(n_clusters=best_k, metric='precomputed').fit(dis_mat)
nmi = sklearn.metrics.normalized_mutual_info_score(labels_true=labels, labels_pred=best_res.labels_)
ari = sklearn.metrics.adjusted_rand_score(labels_true=labels, labels_pred=best_res.labels_)

print("sse_max:", sse_max)
print("best_k:", best_k)
print("nmi:", nmi)
print("ari:", ari)

sse_max: 0.6307978202084701
best_k: 4
nmi: 0.621385371525521
ari: 0.31132584299725
